In [ ]:
import requests
import torch
import torch.nn.functional as F
import torchtext
import torch.nn as nn
from torch.nn import Transformer
import math
from torch import Tensor

url = "https://nlp.stanford.edu/projects/nmt/data/iwslt15.en-vi/"
train_en = [line.split() for line in requests.get(url + "train.en").text.splitlines()]
train_vi = [line.split() for line in requests.get(url + "train.vi").text.splitlines()]
test_en = [line.split() for line in requests.get(url + "tst2013.en").text.splitlines()]
test_vi = [line.split() for line in requests.get(url + "tst2013.vi").text.splitlines()]

In [ ]:
def make_vocab(train_data, min_freq):
    vocab = {}
    for tokenlist in train_data:
        for token in tokenlist:
            if token not in vocab:
                vocab[token] = 0
            vocab[token]+=1
    vocablist = [('<unk>',0), ('<pad>', 0), ('<cls>', 0), ('<eos>', 0)]
    vocabidx = {}
    for token, freq in vocab.items():
        if freq >= min_freq:
            idx = len(vocablist)
            vocablist.append((token, freq))
            vocabidx[token] = idx
    vocabidx['<unk>'] = 0
    vocabidx['<pad>'] = 1
    vocabidx['<cls>'] = 2
    vocabidx['<eos>'] = 3
    return vocablist, vocabidx

vocablist_en, vocabidx_en = make_vocab(train_en, 3)
vocablist_vi, vocabidx_vi = make_vocab(train_vi, 3)

print('vocab size en: ', len(vocablist_en))
print('vocab size vi: ', len(vocablist_vi))

vocab size en:  24420
vocab size vi:  10666


In [ ]:
MODELNAME = 'iwslt15-en-vi-transformer.model'
EPOCH= 15
BATCHSIZE = 16
LR = 0.0001
SRC_VOCAB_SIZE = len(vocablist_en)
TGT_VOCAB_SIZE = len(vocablist_vi)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
BOS_IDX = 2
EOS_IDX = 3
PAD_IDX = 1
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def preprocess(data, vocabidx):
    rr = []
    for tokenlist in data:
        tkl = ['<cls>']
        for token in tokenlist:
            tkl.append(token if token in vocabidx else '<unk>')
        tkl.append('<eos>')
        rr.append(tkl)
    return rr

train_en_prep = preprocess(train_en, vocabidx_en)
train_vi_prep = preprocess(train_vi, vocabidx_vi)
test_en_prep = preprocess(test_en, vocabidx_en)

# for i in range(5):
#     print(train_en_prep[i])
#     print(train_vi_prep[i])
#     print(test_en_prep[i])

In [ ]:
train_data = list(zip(train_en_prep, train_vi_prep))
train_data.sort(key = lambda x: (len(x[0]), len(x[1])))
test_data = list(zip(test_en_prep, test_en, test_vi))

# for i in range(5):
#     print(train_data[i])
# for i in range(5):
#     print(test_data[i])

In [ ]:
def make_batch(data, batchsize):
    bb = [] 
    ben = []
    bvi = []
    for en, vi in data:
        ben.append(en)
        bvi.append(vi)
        if len(ben) >= batchsize:
            bb.append((ben, bvi))
            ben = []
            bvi = []
    if len(ben)>0:
        bb.append((ben, bvi))
    return bb

train_data = make_batch(train_data, BATCHSIZE)
# for i in range(5):
#     print(train_data[i])

In [ ]:
def padding_batch(b):
    maxlen = max([len(x) for x in b])
    for tkl in b:
        for i in range(maxlen - len(tkl)):
            tkl.append('<pad>')

def padding(bb):
    for ben, bvi in bb:
        padding_batch(ben)
        padding_batch(bvi)

padding(train_data)

# for i in range(3):
#     print(train_data[i])

In [ ]:
train_data = [([[vocabidx_en[token] for token in tokenlist] for tokenlist in ben],
               [[vocabidx_vi[token] for token in tokenlist] for tokenlist in bvi]) for ben, bvi in train_data]

test_data = [([vocabidx_en[token] for token in enprep], en, vi) for enprep, en, vi in test_data]

# for i in range(3):
#     print(train_data[i])
# for i in range(3):
#     print(test_data[i])

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [ ]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


# function to create source and destination masks
def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [ ]:
def train():
    transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM).to(DEVICE)
    
    # check if transformer's current parameter dimension > 1 then initialize again
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    optimizer = torch.optim.Adam(transformer.parameters(), lr=LR, betas=(0.9, 0.98), eps=1e-9)
    for epoch in range(EPOCH):
        losses = 0
        step = 0
        for ben, bvi in train_data:
            ben = torch.tensor(ben, dtype=torch.int64).transpose(0,1).to(DEVICE)
            bvi = torch.tensor(bvi, dtype=torch.int64).transpose(0,1).to(DEVICE)

            #remove tail <eos> 
            bvi_input = bvi[:-1, :]

            src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(ben, bvi_input)
            logits = transformer(ben, bvi_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

            optimizer.zero_grad()
            bvi_out = bvi[1:, :]

            #Calculating loss with cross entropy loss function 
            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), bvi_out.reshape(-1))

            #Backward to update all parameters thanks to the loss learned from cross entropy loss 
            loss.backward()
            optimizer.step()
            losses += loss.item()
            if step%100 == 0:
                print('step:', step, ': batchloss', loss.item())
            step += 1
        print('epoch', epoch + 1, ': loss', losses)
    torch.save(transformer.state_dict(), MODELNAME)
train()

step: 0 : batchloss 9.800725936889648
step: 100 : batchloss 5.619462966918945
step: 200 : batchloss 5.459331035614014
step: 300 : batchloss 4.633382320404053
step: 400 : batchloss 4.916737079620361
step: 500 : batchloss 5.051906108856201
step: 600 : batchloss 5.074631214141846
step: 700 : batchloss 4.721857070922852
step: 800 : batchloss 5.464365005493164
step: 900 : batchloss 5.287318229675293
step: 1000 : batchloss 4.8200273513793945
step: 1100 : batchloss 4.69769287109375
step: 1200 : batchloss 4.815011501312256
step: 1300 : batchloss 5.510915279388428
step: 1400 : batchloss 4.508548259735107
step: 1500 : batchloss 5.02527379989624
step: 1600 : batchloss 5.095278739929199
step: 1700 : batchloss 4.732515335083008
step: 1800 : batchloss 4.5647735595703125
step: 1900 : batchloss 4.284892559051514
step: 2000 : batchloss 4.729089260101318
step: 2100 : batchloss 4.752885818481445
step: 2200 : batchloss 4.274634838104248
step: 2300 : batchloss 4.6763529777526855
step: 2400 : batchloss 4.30

In [ ]:
# After decoding, choose the word with the highest probability at each step 
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()
        
        #Returns a tensor array of selected Vietnamese ids 
        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


In [ ]:
def test():
    total = 0
    correct = 0
    model = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM).to(DEVICE)
    model.load_state_dict(torch.load(MODELNAME))
    model.eval()
    ref = []
    pred = []
    for enprep, en, vi in test_data:
        lis = []
        #Get the id of the English sentence as input
        input = torch.tensor([enprep], dtype=torch.int64).transpose(0, 1).to(DEVICE)
        num_tokens = len(enprep)
        src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)

        #Save the decoded result in a variable named tgt_tokens
        tgt_tokens = greedy_decode(
        model,  input, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
        
        #Since the result is in tensor form, we convert it to numpy form, 
        #from there iterate through the ids to get the Vietnamese words corresponding to the id and add it to the list. 
        p = list(tgt_tokens.cpu().numpy())
        for i in p:
            lis.append(list(vocabidx_vi.keys())[list(vocabidx_vi.values()).index(i)])

        print('INPUT', en)
        print('REF', vi)
        print('MT', lis[1:-1])

        #add actual results to the ref list 
        ref.append([vi])
        #Add sentence translation results to list pred 
        pred.append(lis)
    bleu = torchtext.data.metrics.bleu_score(pred, ref)
    print('total:', len(test_data))
    print('bleu:', bleu)

test()

INPUT ['When', 'I', 'was', 'little', ',', 'I', 'thought', 'my', 'country', 'was', 'the', 'best', 'on', 'the', 'planet', ',', 'and', 'I', 'grew', 'up', 'singing', 'a', 'song', 'called', '&quot;', 'Nothing', 'To', 'Envy', '.', '&quot;']
REF ['Khi', 'tôi', 'còn', 'nhỏ', ',', 'Tôi', 'nghĩ', 'rằng', 'BắcTriều', 'Tiên', 'là', 'đất', 'nước', 'tốt', 'nhất', 'trên', 'thế', 'giới', 'và', 'tôi', 'thường', 'hát', 'bài', '&quot;', 'Chúng', 'ta', 'chẳng', 'có', 'gì', 'phải', 'ghen', 'tị', '.', '&quot;']
MT ['Khi', 'tôi', 'còn', 'nhỏ', ',', 'tôi', 'nghĩ', 'đất', 'nước', 'tôi', 'là', 'điều', 'tốt', 'nhất', 'trên', 'hành', 'tinh', ',', 'và', 'tôi', 'lớn', 'lên', 'hát', 'ca', 'hát', 'bài', 'hát', 'có', 'tên', '&quot;', 'Không', 'có', 'gì', 'để', 'làm']
INPUT ['And', 'I', 'was', 'very', 'proud', '.']
REF ['Tôi', 'đã', 'rất', 'tự', 'hào', 'về', 'đất', 'nước', 'tôi', '.']
MT ['Và', 'tôi', 'rất', 'tự', 'hào', 'và', 'rất', 'tự', 'hào', '.']
INPUT ['In', 'school', ',', 'we', 'spent', 'a', 'lot', 'of', 'time',